<a href="https://colab.research.google.com/github/ameympatil/MIT-ML/blob/main/keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%tensorflow_version 
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('Gpu Not found')
print('Found Gpu at {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found Gpu at /device:GPU:0


In [10]:
!pip install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import keras_tuner
from tensorflow import keras
from keras import layers
import pandas as pd
from keras_tuner.tuners import RandomSearch

In [12]:
df = pd.read_csv('/content/Tetuan City power consumption (1).csv')

In [13]:
df.head()

,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
0,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386
1,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434
2,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373
3,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711
4,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964


In [14]:
X = df.drop(columns='Zone 1 Power Consumption')
y = df['Zone 1 Power Consumption']

In [15]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [16]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Power Comsumption')

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [18]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 01m 04s]
val_mean_absolute_error: nan

Best val_mean_absolute_error So Far: nan
Total elapsed time: 00h 06m 15s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [20]:
tuner.search_space_summary()


Search space summary
Default search space size: 20
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_6 (Int)
{'default': None, 'co

In [21]:
tuner.results_summary()

Results summary
Results in project/Power Comsumption
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 14
units_0: 448
units_1: 320
learning_rate: 0.01
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
Score: nan
Trial summary
Hyperparameters:
num_layers: 18
units_0: 160
units_1: 416
learning_rate: 0.0001
units_2: 96
units_3: 288
units_4: 32
units_5: 64
units_6: 96
units_7: 288
units_8: 224
units_9: 32
units_10: 288
units_11: 64
units_12: 448
units_13: 352
units_14: 32
units_15: 32
units_16: 32
units_17: 32
Score: nan
Trial summary
Hyperparameters:
num_layers: 2
units_0: 416
units_1: 192
learning_rate: 0.001
units_2: 416
units_3: 448
units_4: 224
units_5: 96
units_6: 256
units_7: 128
units_8: 512
units_9: 224
units_10: 320
units_11: 384
units_12: 416
units_13: 448
units_14: 448
units_15: 256
units_16: 416
units_17: 224
Score: nan
Trial summary
Hyperparameters:
num_layers: 1